In [24]:
# spc = spotify playlist country
# https://developer.spotify.com/documentation/web-api/reference/search/search/#writing-a-query---guidelines
import requests
import base64
import json
import sys
import pandas as pd
from config import clientId, clientSecret

In [20]:
# Load the CSV file with all music tracks
# to determine unique values, only Track Name and Artist are needed
music_load_df = pd.read_csv("../Music_Data.csv")
music_load_df = music_load_df.loc[:, ["Track Name", "Artist"]]

# Display dataframe header
music_load_df.head()

,Track Name,Artist
0,Starboy,The Weeknd
1,Closer,The Chainsmokers
2,Let Me Love You,DJ Snake
3,Rockabye (feat. Sean Paul & Anne-Marie),Clean Bandit
4,One Dance,Drake


In [21]:
# Drop all of the duplicates from the dataset
music_data_df = music_load_df.drop_duplicates(subset = ["Track Name", "Artist"])

# Export the data into a csv
music_data_df.to_csv("output_data/Music_Data_Tracks.csv")                  
    
# Display unique dataframe
music_data_df

,Track Name,Artist
0,Starboy,The Weeknd
1,Closer,The Chainsmokers
2,Let Me Love You,DJ Snake
3,Rockabye (feat. Sean Paul & Anne-Marie),Clean Bandit
4,One Dance,Drake
...,...,...
320931,No Time (feat. Lil Durk),KSI
320933,People Watching,Conan Gray
320959,Why You No Love Me,John Mayer
321140,Se For Amor,João Gomes


In [22]:
# Create DF to search
search_trackID_df = music_data_df.copy()

# Utilized this code to create a sample test set to run through API
#search_trackID_df = search_trackID_df.iloc[:5]

# Display dataframe header
search_trackID_df.head()

,Track Name,Artist
0,Starboy,The Weeknd
1,Closer,The Chainsmokers
2,Let Me Love You,DJ Snake
3,Rockabye (feat. Sean Paul & Anne-Marie),Clean Bandit
4,One Dance,Drake


In [23]:
# Step 1 - Authorization 
url = "https://accounts.spotify.com/api/token"
headers = {}
data = {}
## O.auth customization for spotify api
# Encode as Base64
message = f"{clientId}:{clientSecret}"
messageBytes = message.encode('ascii')
base64Bytes = base64.b64encode(messageBytes)
base64Message = base64Bytes.decode('ascii')

headers['Authorization'] = f"Basic {base64Message}"
data['grant_type'] = "client_credentials"

## end of setup for O.Auth... line 23 begins actual use of O.Auth
r = requests.post(url, headers=headers, data=data)
# token will expire in an hour but since this is not an industrial sized app I'm not 
# going to stress about pulling in an expiration and will get a token each time the script runs
token = r.json()['access_token']
print(token)

BQBKyR5-HOsUMnOdrv7YtG_ooJFJppbLJh-8Wq8N2Ow4s5ypoM_HD65DWKmZcoNf7TXBrABlPbvkkxUpy7NEn_eR7Fw4haO0OFxngWVCc7VzG3pjRco


In [5]:
# Search spotify based on csv file
params = {"type": "track", 
          "limit": 1}    

searchUrl='https://api.spotify.com/v1/search'
request_header = {
    "Authorization": "Bearer " + token
}

# Print to show API search has started
print("Starting Track ID search")

# Loop through all iterations within the dataframe
for index, row in search_trackID_df.iterrows():
         
    # Get search terms
    track = row["Track Name"]
    artist = row["Artist"]

    # Add filter parameters
    params["q"] = f"{track}+{artist}"

    # Build URL and convert to a JSON
    search_response = requests.get(url=searchUrl, headers=request_header, params=params).json()
    #print(json.dumps(search_response, indent=4, sort_keys=True))

    # Add Track ID data from Spotify into dataframe
    try:
        search_trackID_df.loc[index, "Track ID"] = search_response["tracks"]["items"][0]["id"]
        search_trackID_df.loc[index, "Popularity"] = search_response["tracks"]["items"][0]["popularity"]
    except (KeyError, IndexError):
        search_trackID_df.loc[index, "Track ID"] = "No Track Found"
        search_trackID_df.loc[index, "Popularity"] = "No Data Found"
        
    #log the results - may want to comment this section out when entire list is run
    print(f"{search_trackID_df.loc[index, 'Track Name']} - Track ID: {search_trackID_df.loc[index, 'Track ID']}")


# Print to show API search has ended
print("Track ID search Complete")

# Export the data into a csv
search_trackID_df.to_csv("output_data/Track_ID_Data.csv")
                    
# Display the sample data
search_trackID_df.head()
          

Starting Track ID search
Merry Christmas, Happy Holidays - Track ID: 4v9WbaxW8HdjqfUiWYWsII
Bande organisée - Track ID: 2JNJSm3CjesTWLOcJm4Kiz
Extasy - Track ID: 79lc69ZXfL4WmJIzVSDmWT
High Life - Track ID: 0Vy7fWzknPBpNTwJkGc7Sm
Millionär - Track ID: 6mw02h3qbWmuq2bJCRjRAe
Mit den Jungz - Track ID: 3aDYo34VzVf2i8aJyJk0ca
Vielen Dank - Track ID: 4xN9cgMVhYyhnugUFZdhcZ
4:00 AM - Track ID: 1nX9KhK3Fff27SnrIor2Yb
Bigger Than You (feat. Drake & Quavo) - Track ID: 5S1IUPueD0xE0vj4zU3nSf
Good Drank - Track ID: 39pS70eeDvyCAF3t8NAlVV
It's A Vibe - Track ID: 6H0AwSQ20mo62jGlPGB8S6
Momma I Hit A Lick (feat. Kendrick Lamar) - Track ID: 7jKUnzzOGhAlPjr7LkIUlE
Rule The World (feat. Ariana Grande) - Track ID: 1R6VwZ8TuHRzxZUxe88n4I
Whip (feat. Travis Scott) - Track ID: 6Z4rmc0uujCpl8yXe3yjgI
1.5 - Track ID: 2wOXxtHZgRkkrkEbKLzzqs
4L - Track ID: 1uULQtJ8rbsX5TwjPpPc9Q
a lot - Track ID: 2t8yVaLvJ0RenpXUIAC52d
a&t - Track ID: 6ho88pvz4c8WCSFbdVXNia
all my friends - Track ID: 4paAidoE8VAemwU4jgCc6l
asm

Dancing Queen - Track ID: 0GjEhVFGZW8afUYGChu3Rr
Happy New Year - Track ID: 4AsvGVDWs16fqIiIdDzyvX
Loco Enamorado - Track ID: 4N1c5rmWOzRmB9SMacr5wB
Bloody Waters (with Anderson .Paak & James Blake) - Track ID: 4KXwFI9pgJLpUIAc9oSL8j
Back In Black - Track ID: 08mG3Y1vljYA6bvDt4Wqkj
Highway to Hell - Track ID: 2zYzyRzz6pRmhPzyfMEC8s
Realize - Track ID: 4JDjZ2AzlHnde5uUy0TveV
Shot In The Dark - Track ID: 0sfdiwck2xr4PteGOdyOfz
You Shook Me All Night Long - Track ID: 2SiXAy7TuUkycRVbbWDEpo
Hello - Track ID: 62PaSfnXSMyLshYJrlTuL3
Make You Feel My Love - Track ID: 5FgPwJ7Nh2FVmIXviKl2VF
Send My Love (To Your New Lover) - Track ID: 0t7fVeEJxO2Xi4H2K5Svc9
Someone Like You - Track ID: 1zwMYTA5nlNjZxYrvBB2pV
Water Under the Bridge - Track ID: 4jL6WWKFDqCOPo2hC3VhSS
When We Were Young - Track ID: 7GgWAITsYJaRM3r50rfh5w
Next Level - Track ID: 2zrhoHlFKxFTRF5aMyxMoQ
Hero - Track ID: 29p6UW08MXzwj9thWBW3uF
28 - Track ID: 3pGNUEIlaUlCU8mXxKInzw
Burn It - Track ID: 2hX8rtWyNxYyaBT64H3LYU
Daechwita -

The Ultracheese - Track ID: 3PhbzvTllkq1rkULafc6ns
The World's First Ever Monster Truck Front Flip - Track ID: 5rqQTEIVK2PTuXU9GI2wT0
Tranquility Base Hotel & Casino - Track ID: 2URDbWGmPz3vhagl25p8OC
Why'd You Only Call Me When You're High? - Track ID: 086myS9r57YsLbJpU0TgK9
(You Make Me Feel Like) A Natural Woman - Track ID: 5K09WxKdlkroDyEVyOSySy
Chain of Fools - Track ID: 2oZmMp5M6L0Rh7G84Um2tF
I Say a Little Prayer - Track ID: 3NfxSdJnVdon1axzloJgba
Respect - Track ID: 7s25THrKz86DM225dOYwnr
Son of a Preacher Man - Track ID: 4uCbetXW9LStYPUjCpBVp9
Think - Track ID: 4yQw7FR9lcvL6RHtegbJBh
34+35 - Track ID: 6Im9k8u9iIzKMrmV7BWtlF
34+35 Remix (feat. Doja Cat, Megan Thee Stallion) - Remix - Track ID: 2sQH02O2HBCPJCztBrCWNa
7 rings - Track ID: 6ocbgoVGwYJhOv1GgI9NsF
7 rings (feat. 2 Chainz) - Remix - Track ID: 6AyI8UGx8Y4peb7pLOy2pf
bad idea - Track ID: 5Il6Oe7lr5XM7A0cWbVQtr
Bad To You (with Normani & Nicki Minaj) - Track ID: 7vJS1DPc3FzBtqBs8n3mW5
better off - Track ID: 3NbTQ8ZbHU6MS

Bichiyal - Track ID: 4j4w4CXm6BSr0s25wAWrrX
BOOKER T - Track ID: 26w9NTiE9NGjW1ZvIOd1So
BYE ME FUI - Track ID: 0FktnzDqjClqU6iqOZVEs2
Callaita - Track ID: 71wFwRo8xGc4lrcyKwsvba
Caro - Track ID: 6it15CsDlkqB7N4lF0C1qM
Chambea - Track ID: 1Bqg4yFeVDJxchh6MjkGKy
DÁKITI - Track ID: 47EiUVwUp4C9fGccaPuUCS
De Museo - Track ID: 267NGliXM8YLVZiKAD9Otm
Dime Si Te Acuerdas - Track ID: 3kG1jgWIDmjJkXhOiOmioO
EL MUNDO ES MÍO - Track ID: 36DHxTW2xdr9GG15T9oK9L
EN CASITA - Track ID: 2DhNdoHxavs7Qtf5MnhZD0
Está Cabrón Ser Yo - Track ID: 4saHEjXw7O1NXJOzBkUKKG
Estamos Arriba - Track ID: 0bCPJsVGDxPYwQMDZMi4NW
Estamos Bien - Track ID: 2OWVCFTolecLiGZPquvWvT
Hablamos Mañana - Track ID: 1V8XneTrX6kKeoxYbk4x6h
HACIENDO QUE ME AMAS - Track ID: 26QApq0cuO9brJigr0C0W4
HOY COBRÉ - Track ID: 0tjZv2hChdHZCW1zFXpy1J
Ignorantes - Track ID: 38RIEav39FGi86jppxO0du
La Difícil - Track ID: 6NfrH0ANGmgBXyxgV2PeXt
LA DROGA - Track ID: 67ACYA3UI6QMNtHX6b34wX
LA NOCHE DE ANOCHE - Track ID: 2XIc1pqjXV3Cr2BQUGNBck
La Santa

Sunday Morning Jetpack - Track ID: 5VlstKgCJPHDIMCQO4jHcQ
Voices In My Head/Stick To The Plan - Track ID: 2pzuU8vLaG8W1P31X65ZP9
Why Would I Stop? - Track ID: 46hDiH4YvAWXE04GskkIHS
Wolves (feat. Post Malone) - Track ID: 33gwZOGJWEZ7dRWPqPxBEZ
ZTFO - Track ID: 0z36JaXXoFXuPuQucKEsdx
Man's Not Hot - Track ID: 6X5OFBbrsHRsyO1zP7udgr
Tigers - Track ID: 7jnaow1bGpKO2VkgtYE8Vv
Ain't No Sunshine - Track ID: 1k1Bqnv2R0uJXQN4u6LKYt
Lean on Me - Track ID: 3M8FzayQWtkvOhqMn2V4T2
Lovely Day - Track ID: 0bRXwKfigvpKZUurwqAlEh
8 - Track ID: 6X29iaaazwho3ab7GNue5r
all the good girls go to hell - Track ID: 6IRdLKIyS4p7XNiP8r6rsx
bad guy - Track ID: 2Fxmhks0bxGSBdJ92vM42m
bad guy (with Justin Bieber) - Track ID: 3yNZ5r3LKfdmjoS3gkhUCT
bellyache - Track ID: 51NFxnQvaosfDDutk0tams
bury a friend - Track ID: 4SSnFejRGlZikf02HLewEF
come out and play - Track ID: 7wC5eZcFS1Q1BsQ35DU6H4
everything i wanted - Track ID: 3ZCTVFBt2Brf31RLEnCkWJ
goodbye - Track ID: 3LgWsmilsrWXiPYQFRD0T7
i love you - Track ID: 6Cc

...

The Business - Track ID: 6f3Slt0GbA2bPZlz0aIFXN
Entre Nosotros - Track ID: 5LazbgCsa2IskJbPMnI8Qd
Rita - Track ID: 6NEJZvJeu2pIMpRbiyhCPi
Seek Bromance - Avicii Vocal Edit - Track ID: 5BIgbz2Dy7X2I0bnkKH9Mb
Miénteme - Track ID: 6ebo7CQn4ILLGvXRmXHhQz
Oye - Track ID: 6Jsfaw9b0hopNZ1SVHEXmW
Body (Remix) [feat. ArrDee, E1 (3x3), ZT (3x3), Bugzy Malone, Buni, Fivio Foreign & Darkoo] - Track ID: 6uvMKqNlrSvcC4NaKnrwjZ
GOODMORNINGTOKYO! - Track ID: 32aVpvig4eQuNjMzQ8pNKN
Little Bit of Love - Track ID: 3Iba59sFmGhQ4kbCY5L8AX
Another Love - Track ID: 3JvKfv6T31zO0ini8iNItO
Free Fallin' - Track ID: 5tVA6TkbaAH9QMITTQRrNv
I Won't Back Down - Track ID: 7gSQv1OHpkIoAdUiRLdmI6
Runnin' Down A Dream - Track ID: 5yAu4njFSdM47dfsTkQPZ2
Time To Move On - Track ID: 20dt11ZTAtjsfcNJLP9u30
Wildflowers - Track ID: 2Pr1nZpt8A8WP7QYpyq6L3
You Don't Know How It Feels - Track ID: 7MooGz4ZPE4bNxjFegR6Jx
You Wreck Me - Track ID: 0YkbYk24ODhuewb79zZZzM
American Girl - Track ID: 7MRyJPksH3G2cXHN8UKYzP
Breakdown - T

Sangría - con WOS - Track ID: 1fJjC8C0BdgUHlKIP2ODQf
Bandito - Track ID: 5INKIG4QNuD4xq7qZcfPzp
Bounce Man - Track ID: 2GkmE8BrD00ngyjs2PT9Fp
Chlorine - Track ID: 23OXdR7YuUBVWh5hSnYJau
Choker - Track ID: 732E6ibFkQR5lNIPEeiEnx
Cut My Lip - Track ID: 6gFtgkh8ShkrJn52Cbdhxy
Formidable - Track ID: 2jfyY5GIc7wsYVIrCnjblC
Good Day - Track ID: 3uwYgNxFDfx1GoLB6tLoUn
Heathens - Track ID: 6i0V12jOa3mr6uu4WYhUBr
Jumpsuit - Track ID: 1E1uGhNdBe6Dddbgs2KqtZ
Leave the City - Track ID: 5HCXMBxZIVaVLXpyoqPM3g
Legend - Track ID: 1CDVadnneswMxi6gBqJTtC
Level of Concern - Track ID: 6xZ4Q2k2ompmDppyeESIY8
Levitate - Track ID: 2uBhGx3m5iJrtsql8Qk00j
Morph - Track ID: 01vv2AjxgP4uUyb8waYO5Y
Mulberry Street - Track ID: 19kX6hSlYH31js2SL4jgrj
My Blood - Track ID: 5HeKOKc4phmLn8e4I7EkzD
Neon Gravestones - Track ID: 0sXvjZV7p4uvyPN4uQo3FH
Never Take It - Track ID: 1edgaGx2x2AQeWtL2OPEMW
Nico And The Niners - Track ID: 5SehvGGC53A7SZKCLXQcyt
No Chances - Track ID: 2OdfQjIJlomZFUBTEDKMre
Pet Cheetah - Track ID

Mi Niña - Track ID: 2bgcUk2A3jjKbCJ7KPquTi
Vacaciones - Track ID: 1rXojdsUqqxGj2WCmJGWHP
Aullando - Track ID: 7BuHdm4lQqcyG3qTwdkxus
Reggaetón en lo Oscuro - Track ID: 7LLXLd0pisbEAdIVK502VM
Fr Fr (feat. Lil Skies) - Track ID: 1RagpNpkD3KcjXMLGMe99V
Hopeless Romantic (feat. Swae Lee) - Track ID: 7IBSffWIu7P2MC7kMwy2FM
Hot Now - Track ID: 4Q3HHwkLVwHrSVad0RUz3V
Letterman - Track ID: 4EsYkJjHKMejYLp54woB9c
Ocean - Track ID: 3qNTqHjFfyJzMgZoYcUqjV
Real Rich (feat. Gucci Mane) - Track ID: 5t8djD6oZcfAyqF4cTQfFb
Rolling Papers 2 - Track ID: 3Ulh8AlXmYztutPMN59e0s
Something New (feat. Ty Dolla $ign) - Track ID: 6u8hSdYZxbK5EczDPRIXXf
Come Closer (feat. Drake) - Track ID: 2zYs4BonN2ydkbrRk333SN
I Wish It Could Be Christmas Everyday - Track ID: 1mH6tZC6iXIHPI6EixCwxw
Bella - Track ID: 68sQIehHsGohLMgcP5nlND
Bella - Remix - Track ID: 78nLeZtgvDeKygduWsgeUk
CANGURO - Track ID: 2UAWYufVaO3EnUUNzl0ppO
Não Pode Se Apaixonar - Track ID: 2z1xnmmPMKe7rF9zjsI7gj
#PROUDCATOWNERREMIX - Track ID: 5dkZ2Drc

Largado As Traças - Acústico - Track ID: 74gguwbBvfBTurNJUWFOsC
Notificação Preferida - Ao Vivo - Track ID: 3tA0vALUwrzkTgHcm9j6oJ
Letícia - Track ID: 4O7EchQjAhOGVgQqBAu7mi
Chicken Fried - Track ID: 0LQtEJt7x0s6knb6RKdRYc
Rewrite The Stars - Track ID: 65fpYBrI8o2cfrwf2US4gq
Redemption (with Babes Wodumo) - Track ID: 76iVOVsliCHlWqKuDnCfhE
Redemption Interlude - Track ID: 4FpfU1O7WCLBnmwu5XAFk4
JU$T (feat. Pharrell Williams & Zack de la Rocha) - Track ID: 0YdjfAmUyxF4Dh0HNEhmoI
Ain't My Fault - Track ID: 0ADG9OgdVTL7fgREP75BrZ
All the Time - Track ID: 3OTSBoYKO7HzGVj8Bu2OH9
Don't Worry Bout Me - Track ID: 507tQXutCPYNFkpOq8tmKF
I Would Like - Track ID: 4Q4jmPHwu0wrJvqrld0FQ6
Love Me Land - Track ID: 5knSvwiFenQOkDoriHi2Rg
Lush Life - Track ID: 1rIKgCH4H52lrvDcz50hS8
Never Forget You - Track ID: 6hmhG1b4LEyNuashVvuIAo
Only You - Track ID: 5SleX0NFeODNY7Xenal3gr
Ruin My Life - Track ID: 4nyY8oVjbX2d4qzlpiVM5n
So Good (feat. Ty Dolla $ign) - Track ID: 54LGxQGf5LPXNGjtzDZ5IE
Talk About Lov

,Track Name,Artist,Track ID,Popularity
0,"Merry Christmas, Happy Holidays",*NSYNC,4v9WbaxW8HdjqfUiWYWsII,81.0
1,Bande organisée,13 Organisé,2JNJSm3CjesTWLOcJm4Kiz,59.0
2,Extasy,187 Strassenbande,79lc69ZXfL4WmJIzVSDmWT,60.0
3,High Life,187 Strassenbande,0Vy7fWzknPBpNTwJkGc7Sm,50.0
4,Millionär,187 Strassenbande,6mw02h3qbWmuq2bJCRjRAe,63.0


In [8]:
# Reloading track data to re-run in spotify - did not want to rerun the API therefore pulling in the csv file
search_trackID_df = pd.read_csv("output_data/Track_ID_Data.csv")

# Display dataframe header
search_trackID_df.head()

,Unnamed: 0,Track Name,Artist,Track ID,Popularity
0,0,"Merry Christmas, Happy Holidays",*NSYNC,4v9WbaxW8HdjqfUiWYWsII,81.0
1,1,Bande organisée,13 Organisé,2JNJSm3CjesTWLOcJm4Kiz,59.0
2,2,Extasy,187 Strassenbande,79lc69ZXfL4WmJIzVSDmWT,60.0
3,3,High Life,187 Strassenbande,0Vy7fWzknPBpNTwJkGc7Sm,50.0
4,4,Millionär,187 Strassenbande,6mw02h3qbWmuq2bJCRjRAe,63.0


In [9]:
# Duplicate Dataframe for additional data searching
search_audiodata_df = search_trackID_df.copy()

# Display dataframe header
search_audiodata_df.head()

,Unnamed: 0,Track Name,Artist,Track ID,Popularity
0,0,"Merry Christmas, Happy Holidays",*NSYNC,4v9WbaxW8HdjqfUiWYWsII,81.0
1,1,Bande organisée,13 Organisé,2JNJSm3CjesTWLOcJm4Kiz,59.0
2,2,Extasy,187 Strassenbande,79lc69ZXfL4WmJIzVSDmWT,60.0
3,3,High Life,187 Strassenbande,0Vy7fWzknPBpNTwJkGc7Sm,50.0
4,4,Millionär,187 Strassenbande,6mw02h3qbWmuq2bJCRjRAe,63.0


In [16]:
# Now that we have the track ID - we can get track data from Spotify

print("Starting Audio Features search")

for index, row in search_audiodata_df.iterrows():
    
    # Track ID needed for Audio Features URL search
    track_id = row["Track ID"]
    
    audioUrl = f"https://api.spotify.com/v1/audio-features/{track_id}"
    request_header = {"Authorization": "Bearer " + token}

    audio_response = requests.get(url=audioUrl, headers=request_header).json()
    #print(json.dumps(audio_response, indent=4, sort_keys=True))

    # add audio features from Spotify into dataframe
    try:
        search_audiodata_df.loc[index, "acousticness"] = audio_response["acousticness"]
        search_audiodata_df.loc[index, "danceability"] = audio_response["danceability"]
        search_audiodata_df.loc[index, "duration_ms"] = audio_response["duration_ms"]
        search_audiodata_df.loc[index, "energy"] = audio_response["energy"]
        search_audiodata_df.loc[index, "instrumentalness"] = audio_response["instrumentalness"]
        search_audiodata_df.loc[index, "key"] = audio_response["key"]
        search_audiodata_df.loc[index, "liveness"] = audio_response["liveness"]
        search_audiodata_df.loc[index, "loudness"] = audio_response["loudness"]
        search_audiodata_df.loc[index, "mode"] = audio_response["mode"]
        search_audiodata_df.loc[index, "speechiness"] = audio_response["speechiness"]
        search_audiodata_df.loc[index, "tempo"] = audio_response["tempo"]
        search_audiodata_df.loc[index, "time_signature"] = audio_response["time_signature"]
        search_audiodata_df.loc[index, "valence"] = audio_response["valence"]
    except (KeyError, IndexError):
        search_audiodata_df.loc[index, "acousticness"] = "No Audio Data Found"
        search_audiodata_df.loc[index, "danceability"] = "No Audio Data Found"
        search_audiodata_df.loc[index, "duration_ms"] = "No Audio Data Found"
        search_audiodata_df.loc[index, "energy"] = "No Audio Data Found"
        search_audiodata_df.loc[index, "instrumentalness"] = "No Audio Data Found"
        search_audiodata_df.loc[index, "key"] = "No Audio Data Found"
        search_audiodata_df.loc[index, "liveness"] = "No Audio Data Found"
        search_audiodata_df.loc[index, "loudness"] = "No Audio Data Found"
        search_audiodata_df.loc[index, "mode"] = "No Audio Data Found"
        search_audiodata_df.loc[index, "speechiness"] = "No Audio Data Found"
        search_audiodata_df.loc[index, "tempo"] = "No Audio Data Found"
        search_audiodata_df.loc[index, "time_signature"] = "No Audio Data Found"
        search_audiodata_df.loc[index, "valence"] = "No Audio Data Found"
    
    # log the results - may want to comment this section out when entire list is run
    #print(f"{search_audiodata_df.loc[index, 'Track Name']} - Audio Data Recorded")


# Print to show API search has ended
print("Audio Features search Complete")    
    
# Export the data into a csv
search_audiodata_df.to_csv("output_data/Audio_Features_Data.csv")

#Display data
search_audiodata_df.head()


Starting Audio Features search
Audio Features search Complete


,Unnamed: 0,Track Name,Artist,Track ID,Popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,0,"Merry Christmas, Happy Holidays",*NSYNC,4v9WbaxW8HdjqfUiWYWsII,81.0,0.104,0.643,255307.0,0.939,0.0,5.0,0.881,-3.967,1.0,0.0463,104.999,4.0,0.756
1,1,Bande organisée,13 Organisé,2JNJSm3CjesTWLOcJm4Kiz,59.0,0.117,0.901,356347.0,0.939,0.0,6.0,0.0643,-2.762,1.0,0.274,142.948,4.0,0.805
2,2,Extasy,187 Strassenbande,79lc69ZXfL4WmJIzVSDmWT,60.0,0.0289,0.863,174021.0,0.739,0.0256,9.0,0.0931,-5.879,0.0,0.153,129.964,4.0,0.547
3,3,High Life,187 Strassenbande,0Vy7fWzknPBpNTwJkGc7Sm,50.0,0.0426,0.65,241425.0,0.815,0.0,2.0,0.327,-5.192,1.0,0.379,95.464,4.0,0.452
4,4,Millionär,187 Strassenbande,6mw02h3qbWmuq2bJCRjRAe,63.0,0.176,0.72,159212.0,0.789,0.000006,1.0,0.138,-5.255,1.0,0.339,93.001,4.0,0.226


In [40]:
# Make copy of dataframe and keep only first 2000 lines since data stopped generating after that point
audiodata_one_df = search_audiodata_df[search_audiodata_df["Unnamed: 0"] < 2001]
    
# Export the data into a csv
audiodata_one_df.to_csv("output_data/AudioOne.csv")

#Display DF
audiodata_one_df.tail()

,Unnamed: 0,Track Name,Artist,Track ID,Popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
1996,1996,ROCKSTAR BIKERS & CHAINS,Gunna,2UrPiYObE8itBR1U44Zu3A,47.0,0.00481,0.89,129320.0,0.564,0.0,10.0,0.312,-9.285,0.0,0.23,120.029,4.0,0.332
1997,1997,SKYBOX,Gunna,4jMPZl0GrSFgbogJ0C6sNG,53.0,0.132,0.78,178093.0,0.635,0.0,1.0,0.1,-7.917,1.0,0.17,140.997,4.0,0.371
1998,1998,Speed It Up,Gunna,6PfI6EeLNiNGVAVg11nv7D,50.0,0.0394,0.912,179022.0,0.587,0.01,7.0,0.111,-7.395,1.0,0.0582,143.027,4.0,0.435
1999,1999,TOP FLOOR (feat. Travis Scott),Gunna,4YhJ3a6kBPGJWX4Ek77sDd,64.0,0.206,0.823,169907.0,0.504,0.0,8.0,0.417,-8.65,0.0,0.16,112.985,4.0,0.481
2000,2000,Wit It,Gunna,1unFtbwcAnSGgYPmp52vc0,52.0,0.197,0.868,139644.0,0.657,0.0,7.0,0.105,-5.318,0.0,0.0745,125.905,4.0,0.508


In [7]:
# Since File did not return completed results, will load csv to rerun "No Audio Data Found" tracks back through
# I suspect issue may be with the amount of data being called from API
search_audiodata_two_df = pd.read_csv("output_data/Audio_Features_Data.csv")

# Display dataframe
search_audiodata_two_df

,Unnamed: 0.1,Unnamed: 0,Track Name,Artist,Track ID,Popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,0,0,"Merry Christmas, Happy Holidays",*NSYNC,4v9WbaxW8HdjqfUiWYWsII,81.0,0.104,0.643,255307.0,0.939,0.0,5.0,0.881,-3.967,1.0,0.0463,104.999,4.0,0.756
1,1,1,Bande organisée,13 Organisé,2JNJSm3CjesTWLOcJm4Kiz,59.0,0.117,0.901,356347.0,0.939,0.0,6.0,0.0643,-2.762,1.0,0.274,142.948,4.0,0.805
2,2,2,Extasy,187 Strassenbande,79lc69ZXfL4WmJIzVSDmWT,60.0,0.0289,0.863,174021.0,0.739,0.0256,9.0,0.0931,-5.879,0.0,0.153,129.964,4.0,0.547
3,3,3,High Life,187 Strassenbande,0Vy7fWzknPBpNTwJkGc7Sm,50.0,0.0426,0.65,241425.0,0.815,0.0,2.0,0.327,-5.192,1.0,0.379,95.464,4.0,0.452
4,4,4,Millionär,187 Strassenbande,6mw02h3qbWmuq2bJCRjRAe,63.0,0.176,0.72,159212.0,0.789,6.4e-06,1.0,0.138,-5.255,1.0,0.339,93.001,4.0,0.226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5653,5653,5653,Otra vez (feat. J Balvin),Zion & Lennox,3QwBODjSEzelZyVjxPOHdq,77.0,No Audio Data Found,No Audio Data Found,No Audio Data Found,No Audio Data Found,No Audio Data Found,No Audio Data Found,No Audio Data Found,No Audio Data Found,No Audio Data Found,No Audio Data Found,No Audio Data Found,No Audio Data Found,No Audio Data Found
5654,5654,5654,A Million Dreams,Ziv Zaifman,0RoA7ObU6phWpqhlC9zH4Z,74.0,No Audio Data Found,No Audio Data Found,No Audio Data Found,No Audio Data Found,No Audio Data Found,No Audio Data Found,No Audio Data Found,No Audio Data Found,No Audio Data Found,No Audio Data Found,No Audio Data Found,No Audio Data Found,No Audio Data Found
5655,5655,5655,Control,Zoe Wees,50Td3qilgs8BLtv8mHyT1t,75.0,No Audio Data Found,No Audio Data Found,No Audio Data Found,No Audio Data Found,No Audio Data Found,No Audio Data Found,No Audio Data Found,No Audio Data Found,No Audio Data Found,No Audio Data Found,No Audio Data Found,No Audio Data Found,No Audio Data Found
5656,5656,5656,Girls Like Us,Zoe Wees,5SFXOMJJ334Wn2PwBHeRZN,61.0,No Audio Data Found,No Audio Data Found,No Audio Data Found,No Audio Data Found,No Audio Data Found,No Audio Data Found,No Audio Data Found,No Audio Data Found,No Audio Data Found,No Audio Data Found,No Audio Data Found,No Audio Data Found,No Audio Data Found


In [11]:
# Make copy of dataframe and drop first 2000 lines since data stopped generating after that point
second_search = search_audiodata_two_df[search_audiodata_two_df["Unnamed: 0"] > 2000]

# Drop all of the "No Audio Data Found" columns
second_search = second_search.loc[:, ["Track Name", "Artist", "Track ID", "Popularity"]]

# Display dataframe
second_search.head()

,Track Name,Artist,Track ID,Popularity
2001,World Is Yours (Lil Baby & Gunna),Gunna,0p0jvYdazuWBMC8QacxpUH,52.0
2002,WUNNA,Gunna,0y7Ao3CVJsyKFmqdjutiIF,60.0
2003,A Gente Fez Amor - Ao Vivo,Gusttavo Lima,3tv697PZNnaQN0Mn2zGhS5,59.0
2004,De Menina pra Mulher - Ao Vivo,Gusttavo Lima,7snoDTJeoJ2oVMhf7eoF3t,54.0
2005,Ficha Limpa,Gusttavo Lima,5SwyN9hzmWjBsINlif8li8,66.0


In [12]:
# Now we run next batch of data through API

print("Starting Audio Features search 2nd attempt")

for index, row in second_search.iterrows():
    
    # Track ID needed for Audio Features URL search
    track_id = row["Track ID"]
    
    audioUrl = f"https://api.spotify.com/v1/audio-features/{track_id}"
    request_header = {"Authorization": "Bearer " + token}

    audio_response = requests.get(url=audioUrl, headers=request_header).json()
    #print(json.dumps(audio_response, indent=4, sort_keys=True))

    # add audio features from Spotify into dataframe
    try:
        second_search.loc[index, "acousticness"] = audio_response["acousticness"]
        second_search.loc[index, "danceability"] = audio_response["danceability"]
        second_search.loc[index, "duration_ms"] = audio_response["duration_ms"]
        second_search.loc[index, "energy"] = audio_response["energy"]
        second_search.loc[index, "instrumentalness"] = audio_response["instrumentalness"]
        second_search.loc[index, "key"] = audio_response["key"]
        second_search.loc[index, "liveness"] = audio_response["liveness"]
        second_search.loc[index, "loudness"] = audio_response["loudness"]
        second_search.loc[index, "mode"] = audio_response["mode"]
        second_search.loc[index, "speechiness"] = audio_response["speechiness"]
        second_search.loc[index, "tempo"] = audio_response["tempo"]
        second_search.loc[index, "time_signature"] = audio_response["time_signature"]
        second_search.loc[index, "valence"] = audio_response["valence"]
    except (KeyError, IndexError):
        second_search.loc[index, "acousticness"] = "No Audio Data Found"
        second_search.loc[index, "danceability"] = "No Audio Data Found"
        second_search.loc[index, "duration_ms"] = "No Audio Data Found"
        second_search.loc[index, "energy"] = "No Audio Data Found"
        second_search.loc[index, "instrumentalness"] = "No Audio Data Found"
        second_search.loc[index, "key"] = "No Audio Data Found"
        second_search.loc[index, "liveness"] = "No Audio Data Found"
        second_search.loc[index, "loudness"] = "No Audio Data Found"
        second_search.loc[index, "mode"] = "No Audio Data Found"
        second_search.loc[index, "speechiness"] = "No Audio Data Found"
        second_search.loc[index, "tempo"] = "No Audio Data Found"
        second_search.loc[index, "time_signature"] = "No Audio Data Found"
        second_search.loc[index, "valence"] = "No Audio Data Found"
    
    # log the results - may want to comment this section out when entire list is run
    #print(f"{second_search.loc[index, 'Track Name']} - Audio Data Recorded")
    print(f"{second_search.loc[index, 'Track Name']} - Mode: {second_search.loc[index, 'mode']}")

# Print to show API search has ended
print("Second Audio Features search Complete")    
    
# Export the data into a csv
second_search.to_csv("output_data/Second_Audio_Features_Data.csv")

#Display data
second_search.head()


Starting Audio Features search 2nd attempt
World Is Yours (Lil Baby & Gunna) - Mode: 1.0
WUNNA - Mode: 1.0
A Gente Fez Amor - Ao Vivo - Mode: 0.0
De Menina pra Mulher - Ao Vivo - Mode: 1.0
Ficha Limpa - Mode: 1.0
You Make It Feel Like Christmas (feat. Blake Shelton) - Mode: 0.0
Cupid's Chokehold / Breakfast in America - Mode: 0.0
Stereo Hearts (feat. Adam Levine) - Mode: 1.0
Donuts - Mode: 0.0
Paradies - Mode: 1.0
Verkackt - Mode: 0.0
Vor der Tür - Mode: 0.0
Was erlebt (feat. Bonez MC) - Mode: 1.0
Best Part (feat. Daniel Caesar) - Mode: 1.0
Come Through (feat. Chris Brown) - Mode: 0.0
Conan x Xenia - Mode: 1.0
KMDF - Mode: 1.0
Capital Letters - Mode: 1.0
Let Me Go (with Alesso, Florida Georgia Line & watt) - Mode: 1.0
Most Girls - Mode: 1.0
Starving - Mode: 1.0
Wrong Direction - Mode: 1.0
Gasoline (feat. Taylor Swift) - Mode: 0.0
Little Of Your Love - BloodPop® Remix - Mode: 0.0
Want You Back - Mode: 1.0
3am - Mode: 0.0
Ashley - Mode: 1.0
Bad At Love - Mode: 1.0
Eyes Closed - Mode: 0.0

I'm Yours - Mode: 1.0
Savage Love (Laxed - Siren Beat) - Mode: 0.0
Savage Love (Laxed – Siren Beat) [BTS Remix] - Mode: 0.0
Sweet & Sour (feat. Lauv & Tyga) - Mode: 1.0
Italiana - Mode: 0.0
All Day And Night - Mode: 0.0
All Day And Night - Jax Jones & Martin Solveig Present Europa - Mode: 0.0
Breathe - Mode: 0.0
Harder (with Bebe Rexha) - Mode: 0.0
Instruction - Mode: 0.0
Instruction (feat. Demi Lovato & Stefflon Don) - Mode: 0.0
You Don't Know Me - Mode: 0.0
You Don't Know Me - Radio Edit - Mode: 0.0
King's Dead (with Kendrick Lamar, Future & James Blake) - Mode: 1.0
La Curiosidad - Mode: 0.0
Ni**as In Paris - Mode: 1.0
Numb / Encore - Mode: 1.0
Xerecard - Mode: 1.0
SOLO - Mode: 0.0
Dinero - Mode: 1.0
El Anillo - Mode: 0.0
Ni Tú Ni Yo (feat. Gente de Zona) - Mode: 0.0
Pa Ti - Mode: 0.0
comethru - Mode: 1.0
Mi Cuarto - Mode: 1.0
All I Am - Mode: 1.0
I'll Be There - Mode: 1.0
One Touch - Mode: 0.0
Thursday - Mode: 1.0
3 A.M. - Mode: 1.0
Man With The Bag - Mode: 1.0
CÓMO SE SIENTE - Remi

Higher Higher - Mode: 0.0
Man of the Woods - Mode: 1.0
Midnight Summer Jam - Mode: 0.0
Mirrors - Mode: 0.0
Montana - Mode: 0.0
Morning Light (feat. Alicia Keys) - Mode: 0.0
Sauce - Mode: 1.0
Say Something (feat. Chris Stapleton) - Mode: 1.0
SoulMate - Mode: 0.0
Supplies - Mode: 0.0
Wave - Mode: 1.0
Iko Iko (My Bestie) - Mode: 1.0
ANGELS & DEMONS - Mode: 1.0
Lottery (Renegade) - Mode: 1.0
MORE - Mode: 0.0
POP/STARS - Mode: 1.0
THE BADDEST - Mode: 0.0
Glittery - From The Kacey Musgraves Christmas Show - Mode: 0.0
Royal Rumble - Mode: 1.0
Way down We Go - Mode: 0.0
telepatía - Mode: 0.0
SKRR - Mode: 0.0
Jag trodde änglarna fanns - Mode: 1.0
Be Like That - feat. Swae Lee & Khalid - Mode: 1.0
Heaven - Mode: 1.0
Memory - Mode: 1.0
What Ifs (feat. Lauren Alaina) - Mode: 1.0
All Mine - Mode: 0.0
Closed On Sunday - Mode: 0.0
Every Hour - Mode: 1.0
Everything We Need - Mode: 0.0
Famous - Mode: 1.0
Follow God - Mode: 1.0
Ghost Town - Mode: 0.0
God Is - Mode: 1.0
Hands On - Mode: 0.0
I Love It (& 

Alice - Mode: 0.0
Always Remember Us This Way - Mode: 1.0
Babylon - Mode: 1.0
Bad Romance - Mode: 1.0
Chromatica I - Mode: 1.0
Chromatica II - Mode: 1.0
Enigma - Mode: 1.0
Free Woman - Mode: 1.0
Fun Tonight - Mode: 1.0
I'll Never Love Again - Film Version - Radio Edit - Mode: 1.0
Look What I Found - Mode: 0.0
Million Reasons - Mode: 1.0
Perfect Illusion - Mode: 0.0
Plastic Doll - Mode: 1.0
Poker Face - Mode: 1.0
Rain On Me (with Ariana Grande) - Mode: 1.0
Replay - Mode: 1.0
Shallow - Mode: 1.0
Sine From Above (with Elton John) - Mode: 0.0
Sour Candy (with BLACKPINK) - Mode: 0.0
Stupid Love - Mode: 1.0
The Cure - Mode: 1.0
13 Beaches - Mode: 1.0
Beautiful People Beautiful Problems (feat. Stevie Nicks) - Mode: 1.0
California - Mode: 0.0
Chemtrails Over The Country Club - Mode: 0.0
Cherry - Mode: 0.0
Cinnamon Girl - Mode: 0.0
Coachella - Woodstock In My Mind - Mode: 1.0
Dark But Just A Game - Mode: 1.0
Doin' Time - Mode: 0.0
Fuck it I love you - Mode: 0.0
Groupie Love (feat. A$AP Rocky) -

Sauce It Up - Mode: 0.0
Secure The Bag - Mode: 1.0
Silly Watch - Mode: 1.0
Strawberry Peels (feat. Young Thug & Gunna) - Mode: 0.0
That Way - Mode: 0.0
That Way - Bonus Track - Mode: 0.0
That's a Rack - Mode: 0.0
The Way Life Goes (feat. Nicki Minaj & Oh Wonder) - Remix - Mode: 1.0
The Way Life Goes (feat. Oh Wonder) - Mode: 1.0
Trap This Way (This Way) - Mode: 1.0
Two® - Mode: 1.0
UnFazed (feat. The Weeknd) - Mode: 0.0
Urgency (feat. Syd) - Mode: 1.0
Venetia - Mode: 1.0
Wassup (feat. Future) - Mode: 0.0
X - Mode: 0.0
XO TOUR Llif3 - Mode: 0.0
Yessirskiii - Mode: 1.0
You Better Move - Mode: 1.0
You Was Right - Mode: 1.0
Can't Be Broken - Mode: 0.0
Dark Side Of The Moon (feat. Nicki Minaj) - Mode: 1.0
Dedicate - Mode: 1.0
Demon - Mode: 0.0
Don't Cry (feat. XXXTENTACION) - Mode: 1.0
Dope New Gospel (feat. Nivea) - Mode: 1.0
Dope Niggaz (feat. Snoop Dogg) - Mode: 1.0
Famous (feat. Reginae Carter) - Mode: 1.0
Funeral - Mode: 1.0
Hittas - Mode: 1.0
I Do It (ft. Big Sean, Lil Baby) - Mode: 1

...

In Your Eyes - Mode: No Audio Data Found
In Your Eyes (with Doja Cat) - Remix - Mode: No Audio Data Found
Missed You - Bonus Track - Mode: No Audio Data Found
Nothing Compares - Bonus Track - Mode: No Audio Data Found
Often - Mode: No Audio Data Found
Party Monster - Mode: No Audio Data Found
Pray For Me (with Kendrick Lamar) - Mode: No Audio Data Found
Privilege - Mode: No Audio Data Found
Reminder - Mode: No Audio Data Found
Reminder - Remix - Mode: No Audio Data Found
Repeat After Me (Interlude) - Mode: No Audio Data Found
Rockin’ - Mode: No Audio Data Found
Save Your Tears - Mode: No Audio Data Found
Save Your Tears (with Ariana Grande) (Remix) - Mode: No Audio Data Found
Scared To Live - Mode: No Audio Data Found
Secrets - Mode: No Audio Data Found
Sidewalks - Mode: No Audio Data Found
Six Feet Under - Mode: No Audio Data Found
Snowchild - Mode: No Audio Data Found
Starboy - Mode: No Audio Data Found
The Hills - Mode: No Audio Data Found
Too Late - Mode: No Audio Data Found
True C

The Way - Mode: No Audio Data Found
They Afraid Of You - Mode: No Audio Data Found
This Ain't That - Mode: No Audio Data Found
Throw It Away - Mode: No Audio Data Found
Til The End of Time - Mode: No Audio Data Found
Together - Mode: No Audio Data Found
Topanga - Mode: No Audio Data Found
Toxic Waste - Mode: No Audio Data Found
U Deserve It - Mode: No Audio Data Found
UKA UKA - Mode: No Audio Data Found
Under Enemy Arms - Mode: No Audio Data Found
Who Needs Love - Mode: No Audio Data Found
YELL OH - Mode: No Audio Data Found
Síguelo Bailando - Mode: No Audio Data Found
Bola Rebola - Mode: No Audio Data Found
Bloom - Mode: No Audio Data Found
Dance To This (feat. Ariana Grande) - Mode: No Audio Data Found
Easy - Mode: No Audio Data Found
Lucky Strike - Mode: No Audio Data Found
My My My! - Mode: No Audio Data Found
Plum - Mode: No Audio Data Found
Seventeen - Mode: No Audio Data Found
Take Yourself Home - Mode: No Audio Data Found
The Good Side - Mode: No Audio Data Found
GIANTS - Mode:

Fiel - Remix - Mode: No Audio Data Found
Mi Niña - Mode: No Audio Data Found
Vacaciones - Mode: No Audio Data Found
Aullando - Mode: No Audio Data Found
Reggaetón en lo Oscuro - Mode: No Audio Data Found
Fr Fr (feat. Lil Skies) - Mode: No Audio Data Found
Hopeless Romantic (feat. Swae Lee) - Mode: No Audio Data Found
Hot Now - Mode: No Audio Data Found
Letterman - Mode: No Audio Data Found
Ocean - Mode: No Audio Data Found
Real Rich (feat. Gucci Mane) - Mode: No Audio Data Found
Rolling Papers 2 - Mode: No Audio Data Found
Something New (feat. Ty Dolla $ign) - Mode: No Audio Data Found
Come Closer (feat. Drake) - Mode: No Audio Data Found
I Wish It Could Be Christmas Everyday - Mode: No Audio Data Found
Bella - Mode: No Audio Data Found
Bella - Remix - Mode: No Audio Data Found
CANGURO - Mode: No Audio Data Found
Não Pode Se Apaixonar - Mode: No Audio Data Found
#PROUDCATOWNERREMIX - Mode: No Audio Data Found
$$$ - with Matt Ox - Mode: No Audio Data Found
ALONE, PART 3 - Mode: No Audio

TG4M - Mode: No Audio Data Found
What They Say - Mode: No Audio Data Found
Juju on That Beat (TZ Anthem) - Mode: No Audio Data Found
A Whole New World (End Title) - From "Aladdin" - Mode: No Audio Data Found
Better - Mode: No Audio Data Found
Calamity - Mode: No Audio Data Found
Connexion - Mode: No Audio Data Found
Dusk Till Dawn - Radio Edit - Mode: No Audio Data Found
Entertainer - Mode: No Audio Data Found
Fingers - Mode: No Audio Data Found
I Don’t Wanna Live Forever (Fifty Shades Darker) - From "Fifty Shades Darker (Original Motion Picture Soundtrack)" - Mode: No Audio Data Found
Let Me - Mode: No Audio Data Found
No Candle No Light (feat. Nicki Minaj) - Mode: No Audio Data Found
Outside - Mode: No Audio Data Found
PILLOWTALK - Mode: No Audio Data Found
Sour Diesel - Mode: No Audio Data Found
Still Got Time (feat. PARTYNEXTDOOR) - Mode: No Audio Data Found
Sweat - Mode: No Audio Data Found
There You Are - Mode: No Audio Data Found
Tightrope - Mode: No Audio Data Found
Too Much (f

,Track Name,Artist,Track ID,Popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
2001,World Is Yours (Lil Baby & Gunna),Gunna,0p0jvYdazuWBMC8QacxpUH,52.0,0.113,0.834,157199.0,0.557,0.0,6.0,0.0694,-7.148,1.0,0.543,109.867,4.0,0.575
2002,WUNNA,Gunna,0y7Ao3CVJsyKFmqdjutiIF,60.0,0.0217,0.89,158240.0,0.455,0.0058,0.0,0.11,-9.0,1.0,0.255,148.009,4.0,0.313
2003,A Gente Fez Amor - Ao Vivo,Gusttavo Lima,3tv697PZNnaQN0Mn2zGhS5,59.0,0.366,0.794,237517.0,0.937,0.000004,8.0,0.962,-2.44,0.0,0.0336,134.012,4.0,0.826
2004,De Menina pra Mulher - Ao Vivo,Gusttavo Lima,7snoDTJeoJ2oVMhf7eoF3t,54.0,0.118,0.83,208383.0,0.92,0.000002,4.0,0.234,-2.303,1.0,0.0352,121.982,4.0,0.967
2005,Ficha Limpa,Gusttavo Lima,5SwyN9hzmWjBsINlif8li8,66.0,0.108,0.774,192725.0,0.732,0.0,9.0,0.243,-4.227,1.0,0.0308,135.005,4.0,0.734


In [17]:
# looks like same issue happened again...i believe i'm getting timed out within Spotify 
# Let's try again

# Make copy of dataframe and keep only the lines with actual data
audiodata_two_df = second_search[~second_search["acousticness"].str.contains("No Audio Data Found",  na=False)]
    
# Export the data into a csv
audiodata_two_df.to_csv("output_data/AudioTwo.csv")

#Display DF
audiodata_two_df.tail()

,Track Name,Artist,Track ID,Popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
3428,Que Calor (with J Balvin & El Alfa),Major Lazer,3Wgcni2UwsmJU3wXcnXPe4,3.0,0.000621,0.721,245253.0,0.746,0.699,5.0,0.0825,-6.994,0.0,0.0468,126.039,4.0,0.794
3429,QueLoQue (feat. Paloma Mami),Major Lazer,5jWiuNtsn9mUUEMa4bYYK3,40.0,0.0365,0.649,165176.0,0.812,0.0,9.0,0.0881,-5.086,1.0,0.136,170.047,4.0,0.396
3430,Rave de Favela (feat. BEAM),Major Lazer,1K7WzjKDcDdVbKNGrB0gej,56.0,0.0466,0.891,154154.0,0.894,0.000244,4.0,0.161,-4.595,0.0,0.145,130.045,4.0,0.671
3431,Run Up (feat. PARTYNEXTDOOR & Nicki Minaj),Major Lazer,2q10LFyRsLvgFBaaOmfosx,0.0,0.077,0.768,178031.0,0.862,0.000053,1.0,0.0422,-6.112,0.0,0.057,107.974,4.0,0.65
3432,Sua Cara (feat. Anitta & Pabllo Vittar),Major Lazer,7b4JJ3PG1Ax59EOO7dPBsk,0.0,0.0144,0.402,300250.0,0.321,0.26,5.0,0.105,-16.214,0.0,0.0309,127.969,4.0,0.163


In [6]:
# Make another copy of dataframe and keep only "No Audio Data Found" lines
audiodata_three_df = second_search[second_search["acousticness"].str.contains("No Audio Data Found", na=False)]

#  Drop all of the "No Audio Data Found" lines
third_search = audiodata_three_df.loc[:, ["Track Name", "Artist", "Track ID", "Popularity"]]

# Export data to CSV file
third_search.to_csv("output_data/Third_Search_Set.csv")

# Display dataframe header
third_search.head()

,Track Name,Artist,Track ID,Popularity
1432,"Tied Up (feat. Mr Eazi, RAYE and Jake Gosling)",Major Lazer,1qFSPn37zEeCyRiVpkW9dt,65.0
1433,Titans (feat. Sia & Labrinth),Major Lazer,1PQaukn6h0XlPfqTfPGuDk,6.0
1434,Trigger,Major Lazer,0Z26mbSHJ1dkUu7g9DDgif,53.0
1435,Disco Arranhado - Funk Remix,Malu,3SI736Mng30L5ZTpGEhKEr,63.0
1436,11:00 PM,Maluma,7KbF6AdprOXEEHlsq11Z6d,75.0


In [12]:
# Now we run next batch of data through API

print("Starting Audio Features search 3rd attempt")

for index, row in third_search.iterrows():
    
    # Track ID needed for Audio Features URL search
    track_id = row["Track ID"]
    
    audioUrl = f"https://api.spotify.com/v1/audio-features/{track_id}"
    request_header = {"Authorization": "Bearer " + token}

    audio_response = requests.get(url=audioUrl, headers=request_header).json()
    #print(json.dumps(audio_response, indent=4, sort_keys=True))

    # add audio features from Spotify into dataframe
    try:
        third_search.loc[index, "acousticness"] = audio_response["acousticness"]
        third_search.loc[index, "danceability"] = audio_response["danceability"]
        third_search.loc[index, "duration_ms"] = audio_response["duration_ms"]
        third_search.loc[index, "energy"] = audio_response["energy"]
        third_search.loc[index, "instrumentalness"] = audio_response["instrumentalness"]
        third_search.loc[index, "key"] = audio_response["key"]
        third_search.loc[index, "liveness"] = audio_response["liveness"]
        third_search.loc[index, "loudness"] = audio_response["loudness"]
        third_search.loc[index, "mode"] = audio_response["mode"]
        third_search.loc[index, "speechiness"] = audio_response["speechiness"]
        third_search.loc[index, "tempo"] = audio_response["tempo"]
        third_search.loc[index, "time_signature"] = audio_response["time_signature"]
        third_search.loc[index, "valence"] = audio_response["valence"]
    except (KeyError, IndexError):
        third_search.loc[index, "acousticness"] = "No Audio Data Found"
        third_search.loc[index, "danceability"] = "No Audio Data Found"
        third_search.loc[index, "duration_ms"] = "No Audio Data Found"
        third_search.loc[index, "energy"] = "No Audio Data Found"
        third_search.loc[index, "instrumentalness"] = "No Audio Data Found"
        third_search.loc[index, "key"] = "No Audio Data Found"
        third_search.loc[index, "liveness"] = "No Audio Data Found"
        third_search.loc[index, "loudness"] = "No Audio Data Found"
        third_search.loc[index, "mode"] = "No Audio Data Found"
        third_search.loc[index, "speechiness"] = "No Audio Data Found"
        third_search.loc[index, "tempo"] = "No Audio Data Found"
        third_search.loc[index, "time_signature"] = "No Audio Data Found"
        third_search.loc[index, "valence"] = "No Audio Data Found"
    
    # log the results - may want to comment this section out when entire list is run
    #print(f"{third_search.loc[index, 'Track Name']} - Audio Data Recorded")
    #print(f"{third_search.loc[index, 'Track Name']} - Mode: {third_search.loc[index, 'mode']}")

# Print to show API search has ended
print("Third Audio Features search Complete")    
    
# Export the data into a csv
third_search.to_csv("output_data/AudioThree.csv")

#Display data
#third_search.head()


Starting Audio Features search 3rd attempt
Third Audio Features search Complete


In [25]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
audio_one = pd.read_csv("output_data/AudioOne.csv")
audio_one.drop(columns="Unnamed: 0.1", inplace=True)
audio_one.drop(columns="Unnamed: 0", inplace=True)

# Display sample data
audio_one.head()

,Track Name,Artist,Track ID,Popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,"Merry Christmas, Happy Holidays",*NSYNC,4v9WbaxW8HdjqfUiWYWsII,81.0,0.1040,0.643,255307.0,0.939,0.000000,5.0,0.8810,-3.967,1.0,0.0463,104.999,4.0,0.756
1,Bande organisée,13 Organisé,2JNJSm3CjesTWLOcJm4Kiz,59.0,0.1170,0.901,356347.0,0.939,0.000000,6.0,0.0643,-2.762,1.0,0.2740,142.948,4.0,0.805
2,Extasy,187 Strassenbande,79lc69ZXfL4WmJIzVSDmWT,60.0,0.0289,0.863,174021.0,0.739,0.025600,9.0,0.0931,-5.879,0.0,0.1530,129.964,4.0,0.547
3,High Life,187 Strassenbande,0Vy7fWzknPBpNTwJkGc7Sm,50.0,0.0426,0.650,241425.0,0.815,0.000000,2.0,0.3270,-5.192,1.0,0.3790,95.464,4.0,0.452
4,Millionär,187 Strassenbande,6mw02h3qbWmuq2bJCRjRAe,63.0,0.1760,0.720,159212.0,0.789,0.000006,1.0,0.1380,-5.255,1.0,0.3390,93.001,4.0,0.226


In [26]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
audio_two = pd.read_csv("output_data/AudioTwo.csv")
audio_two.drop(columns="Unnamed: 0", inplace=True)

# Display sample data
audio_two.head()

,Track Name,Artist,Track ID,Popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,World Is Yours (Lil Baby & Gunna),Gunna,0p0jvYdazuWBMC8QacxpUH,52.0,0.1130,0.834,157199.0,0.557,0.000000,6.0,0.0694,-7.148,1.0,0.5430,109.867,4.0,0.575
1,WUNNA,Gunna,0y7Ao3CVJsyKFmqdjutiIF,60.0,0.0217,0.890,158240.0,0.455,0.005800,0.0,0.1100,-9.000,1.0,0.2550,148.009,4.0,0.313
2,A Gente Fez Amor - Ao Vivo,Gusttavo Lima,3tv697PZNnaQN0Mn2zGhS5,59.0,0.3660,0.794,237517.0,0.937,0.000004,8.0,0.9620,-2.440,0.0,0.0336,134.012,4.0,0.826
3,De Menina pra Mulher - Ao Vivo,Gusttavo Lima,7snoDTJeoJ2oVMhf7eoF3t,54.0,0.1180,0.830,208383.0,0.920,0.000002,4.0,0.2340,-2.303,1.0,0.0352,121.982,4.0,0.967
4,Ficha Limpa,Gusttavo Lima,5SwyN9hzmWjBsINlif8li8,66.0,0.1080,0.774,192725.0,0.732,0.000000,9.0,0.2430,-4.227,1.0,0.0308,135.005,4.0,0.734


In [27]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
audio_three = pd.read_csv("output_data/AudioThree.csv")
audio_three.drop(columns="Unnamed: 1", inplace=True)
audio_three.drop(columns="Unnamed: 0", inplace=True)

# Display sample data
audio_three.head()

,Track Name,Artist,Track ID,Popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,"Tied Up (feat. Mr Eazi, RAYE and Jake Gosling)",Major Lazer,1qFSPn37zEeCyRiVpkW9dt,65,0.0385,0.747,166138,0.877,0.000890,9,0.233,-3.782,0,0.0668,107.989,4,0.751
1,Titans (feat. Sia & Labrinth),Major Lazer,1PQaukn6h0XlPfqTfPGuDk,6,0.1010,0.723,199169,0.860,0.000177,7,0.143,-4.891,1,0.0451,94.003,4,0.608
2,Trigger,Major Lazer,0Z26mbSHJ1dkUu7g9DDgif,53,0.1280,0.570,171987,0.585,0.000105,8,0.111,-7.041,1,0.2510,97.285,4,0.378
3,Disco Arranhado - Funk Remix,Malu,3SI736Mng30L5ZTpGEhKEr,63,0.5670,0.746,125560,0.716,0.000000,5,0.062,-2.813,1,0.4090,129.840,4,0.922
4,11:00 PM,Maluma,7KbF6AdprOXEEHlsq11Z6d,75,0.2170,0.777,175733,0.712,0.000000,10,0.091,-4.840,1,0.2770,95.692,4,0.680


In [29]:
# Creating a list of all 3 dataframes that need to be merged into one
frames = [audio_one, audio_two, audio_three]

# Utilize concatinate to merge all three data frames
full_audio = pd.concat(frames)

# Export the data into a csv
full_audio.to_csv("output_data/Full_Audio_Tracks.csv")

# Display completely merged dataframe
full_audio.head()

,Track Name,Artist,Track ID,Popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,"Merry Christmas, Happy Holidays",*NSYNC,4v9WbaxW8HdjqfUiWYWsII,81.0,0.1040,0.643,255307.0,0.939,0.000000,5.0,0.8810,-3.967,1.0,0.0463,104.999,4.0,0.756
1,Bande organisée,13 Organisé,2JNJSm3CjesTWLOcJm4Kiz,59.0,0.1170,0.901,356347.0,0.939,0.000000,6.0,0.0643,-2.762,1.0,0.2740,142.948,4.0,0.805
2,Extasy,187 Strassenbande,79lc69ZXfL4WmJIzVSDmWT,60.0,0.0289,0.863,174021.0,0.739,0.025600,9.0,0.0931,-5.879,0.0,0.1530,129.964,4.0,0.547
3,High Life,187 Strassenbande,0Vy7fWzknPBpNTwJkGc7Sm,50.0,0.0426,0.650,241425.0,0.815,0.000000,2.0,0.3270,-5.192,1.0,0.3790,95.464,4.0,0.452
4,Millionär,187 Strassenbande,6mw02h3qbWmuq2bJCRjRAe,63.0,0.1760,0.720,159212.0,0.789,0.000006,1.0,0.1380,-5.255,1.0,0.3390,93.001,4.0,0.226


In [31]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
music_data_df = pd.read_csv("../Music_data.csv")

# Display sample data
music_data_df.head()

,Position,Track Name,Artist,Streams,Date,Year,Month,Genre,Master Genre
0,1,Starboy,The Weeknd,3135625,1/1/2017,2017,1,canadian pop - canadian contemporary r&b - pop,Pop
1,2,Closer,The Chainsmokers,3015525,1/1/2017,2017,1,pop - pop dance - tropical house - edm - elect...,Pop
2,3,Let Me Love You,DJ Snake,2545384,1/1/2017,2017,1,pop - electronic trap - dance pop - edm - pop ...,Pop
3,4,Rockabye (feat. Sean Paul & Anne-Marie),Clean Bandit,2356604,1/1/2017,2017,1,pop - uk dance - dance pop - uk funky - tropic...,Pop
4,5,One Dance,Drake,2259887,1/1/2017,2017,1,toronto rap - canadian pop - canadian hip hop ...,Hip-Hop


In [35]:
# Merge the two DataFrames together based on the Track Name and Artist they share
music_data_with_spotify = pd.merge(music_data_df, full_audio,  how='left', 
                           left_on=['Track Name','Artist'], right_on = ['Track Name','Artist'])

# Export the data into a csv
music_data_with_spotify.to_csv("../Merged_Music_File_with_Spotify.csv")

music_data_with_spotify.head()

,Position,Track Name,Artist,Streams,Date,Year,Month,Genre,Master Genre,Track ID,...,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,1,Starboy,The Weeknd,3135625,1/1/2017,2017,1,canadian pop - canadian contemporary r&b - pop,Pop,7MXVkk9YMctZqd1Srtv4MB,...,0.587,0.000006,7.0,0.137,-7.015,1.0,0.2760,186.003,4.0,0.486
1,2,Closer,The Chainsmokers,3015525,1/1/2017,2017,1,pop - pop dance - tropical house - edm - elect...,Pop,7BKLCZ1jbUBVqRi2FVlTVw,...,0.524,0.000000,8.0,0.111,-5.599,1.0,0.0338,95.010,4.0,0.661
2,3,Let Me Love You,DJ Snake,2545384,1/1/2017,2017,1,pop - electronic trap - dance pop - edm - pop ...,Pop,0lYBSQXN6rCTvUZvg9S0lU,...,0.716,0.000026,8.0,0.135,-5.371,1.0,0.0349,99.988,4.0,0.163
3,4,Rockabye (feat. Sean Paul & Anne-Marie),Clean Bandit,2356604,1/1/2017,2017,1,pop - uk dance - dance pop - uk funky - tropic...,Pop,5knuzwU65gJK7IF5yJsuaW,...,0.763,0.000000,9.0,0.180,-4.068,0.0,0.0523,101.965,4.0,0.742
4,5,One Dance,Drake,2259887,1/1/2017,2017,1,toronto rap - canadian pop - canadian hip hop ...,Hip-Hop,1zi7xx7UVEFkmKfv06H8x0,...,0.625,0.001800,1.0,0.329,-5.609,1.0,0.0536,103.967,4.0,0.370
